In [2]:
#-- Importing Libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import xml 
 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from bs4 import BeautifulSoup #library for web scraping

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Applications/anaconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.1.1a             |    h1de35cc_1000         3.5 MB  conda-forge
    r-pki-0.1_5.1              |   r351hc070d10_0         104 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    r-openssl-1.0.2            |   r351h46e59ec_1         1.1 MB
    r-mongolite-2.0            |r351h46e59ec_1000         366 KB  conda-forge
    libssh2-1.8.0              |                1         256 KB  conda-forge
    conda-4.6.3                |           py37_0         876 KB  conda-forge
    cryptography-2.5           |   py37ha12b0ac_0         591 KB
    libdb-6.1.26               |    h0a44026

In [3]:
#-- Define the Wikipedia URL from where the data will be extracted
url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url_wiki).text
Canada_Neighbour = BeautifulSoup(source, 'lxml')

In [7]:
#-- create Pandas Dataframe from downloaded data
column_names = ['Postalcode','Borough','Neighborhood']
Toronto_Neigh = pd.DataFrame(columns = column_names)
#-- Determine postcode, borough, neighborhood 
content = Canada_Neighbour.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    Toronto_Neigh = Toronto_Neigh.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#-- Data Cleaning 
Toronto_Neigh = Toronto_Neigh[Toronto_Neigh.Borough!='Not assigned']
Toronto_Neigh = Toronto_Neigh[Toronto_Neigh.Borough!= 0]
Toronto_Neigh.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,Toronto_Neigh.shape[0]):
    if Toronto_Neigh.iloc[i][2] == 'Not assigned':
        Toronto_Neigh.iloc[i][2] = Toronto_Neigh.iloc[i][1]
        i = i+1
                                 
df = Toronto_Neigh.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [8]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [9]:
print(df.shape)
df.head(12)

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
# Install and import Geocoder library
!pip install geocoder
import geocoder
print( 'geocoder imported !')

    100% |████████████████████████████████| 102kB 629kB/s a 0:00:01
geocoder imported !


In [11]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70976500000006, -79.36379132299999]

In [12]:
# Postal codes coordinates
postal_codes = df['Postalcode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [13]:
# Now, we are going to add Latitude and Longitude columns to our DataFrame 
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head(10)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [14]:
#-- Geopy
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
Scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069


In [17]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="Scarborough_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, Toronto are 43.773077, -79.257774.


In [18]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [19]:
CLIENT_ID = 'XNGMCE4ZGZCUHR5F2F20ADU4RXX3P42A3T3EWLYMXIKGEW2U' # your Foursquare ID
CLIENT_SECRET = 'J21VBQV3SQFKCIUHMEI0MR4LEOVO0Y3O4L21S2454TXHASJR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentials:
CLIENT_ID: XNGMCE4ZGZCUHR5F2F20ADU4RXX3P42A3T3EWLYMXIKGEW2U
CLIENT_SECRET:J21VBQV3SQFKCIUHMEI0MR4LEOVO0Y3O4L21S2454TXHASJR


In [20]:
Scarborough_data.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [21]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.81165000000004, -79.19556138899998.


In [22]:
#-- Top 100 venues
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XNGMCE4ZGZCUHR5F2F20ADU4RXX3P42A3T3EWLYMXIKGEW2U&client_secret=J21VBQV3SQFKCIUHMEI0MR4LEOVO0Y3O4L21S2454TXHASJR&v=20180605&ll=43.81165000000004,-79.19556138899998&radius=500&limit=100'

In [23]:
results = requests.get(url).json()

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Scarborough Bad Credit Car Loans,Rental Car Location,43.808123,-79.199133
1,Canadian Appliance Source Whitby,Home Service,43.808353,-79.191331


In [26]:
#-- Neighborhoods in Scarborough
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


In [28]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(97, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811650,-79.195561,Scarborough Bad Credit Car Loans,43.808123,-79.199133,Rental Car Location
1,"Rouge, Malvern",43.811650,-79.195561,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.765690,-79.175299,The Strawberry Patch,43.764738,-79.173081,Tea Room
4,"Guildwood, Morningside, West Hill",43.765690,-79.175299,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


In [29]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 56 uniques categories.


In [30]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Home Service,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Rental Car Location,Restaurant,Sandwich Place,Shanghai Restaurant,Shipping Store,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Train Station,Vietnamese Restaurant,Wings Joint
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
Scarborough_onehot.shape

(97, 57)

In [32]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()

In [33]:
Scarborough_grouped.shape

(16, 57)

In [34]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Agincourt----
                 venue  freq
0          Supermarket  0.12
1        Shopping Mall  0.12
2       Discount Store  0.06
3        Grocery Store  0.06
4  Shanghai Restaurant  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0             Pharmacy   1.0
1          Auto Garage   0.0
2      Badminton Court   0.0
3         Intersection   0.0
4  Japanese Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0               Gym Pool  0.17
1                    Gym  0.17
2                   Park  0.17
3           Skating Rink  0.17
4  General Entertainment  0.17


----Cedarbrae----
                 venue  freq
0           Playground   1.0
1          Auto Garage   0.0
2      Badminton Court   0.0
3         Intersection   0.0
4  Japanese Restaurant   0.0


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0    Coffee Shop   0.2
1         Bakery   0.2
2       Bus Line   0.2
3   Intersectio

In [35]:
#-- Put results into Pandas Dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shopping Mall,Supermarket,Shanghai Restaurant,Park,Chinese Restaurant,Grocery Store,Pool,Vietnamese Restaurant,Department Store,Discount Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,"Birch Cliff, Cliffside West",College Stadium,Gym,General Entertainment,Skating Rink,Park,Gym Pool,Department Store,Grocery Store,Golf Course,Gift Shop
3,Cedarbrae,Playground,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Coffee Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Fast Food Restaurant,Fried Chicken Joint


In [36]:
#-- K-means
# set number of clusters
kclusters = 4

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 3, 0, 0, 0, 0, 0, 0], dtype=int32)

In [37]:
Scarborough_merged = Scarborough_data[0:16]

# add clustering labels
Scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head() # check the last columns!

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561,0,Rental Car Location,Home Service,Construction & Landscaping,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,1,Bar,Wings Joint,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299,0,Gym / Fitness Center,Construction & Landscaping,Tea Room,Park,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
3,M1G,Scarborough,Woburn,43.768216,-79.217610,3,Korean Restaurant,Business Service,Park,Coffee Shop,Wings Joint,Discount Store,Gym,Grocery Store,Golf Course,Gift Shop
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440,0,Playground,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


In [39]:
#-- Examine Clusters

#- Cluster 1
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0,Rental Car Location,Home Service,Construction & Landscaping,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,"Guildwood, Morningside, West Hill",0,Gym / Fitness Center,Construction & Landscaping,Tea Room,Park,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
4,Cedarbrae,0,Playground,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
5,Scarborough Village,0,Train Station,Grocery Store,Indian Restaurant,Restaurant,Wings Joint,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
6,"East Birchmount Park, Ionview, Kennedy Park",0,Discount Store,Department Store,Coffee Shop,Wings Joint,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment
7,"Clairlea, Golden Mile, Oakridge",0,Bus Line,Bakery,Coffee Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Fast Food Restaurant,Fried Chicken Joint
8,"Cliffcrest, Cliffside, Scarborough Village West",0,Fast Food Restaurant,Wings Joint,Pizza Place,Burger Joint,Furniture / Home Store,Liquor Store,Sandwich Place,Coffee Shop,Discount Store,Pharmacy
9,"Birch Cliff, Cliffside West",0,College Stadium,Gym,General Entertainment,Skating Rink,Park,Gym Pool,Department Store,Grocery Store,Golf Course,Gift Shop
11,"Maryvale, Wexford",0,Auto Garage,Intersection,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
12,Agincourt,0,Shopping Mall,Supermarket,Shanghai Restaurant,Park,Chinese Restaurant,Grocery Store,Pool,Vietnamese Restaurant,Department Store,Discount Store


In [40]:
#-- Cluster 2
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",1,Bar,Wings Joint,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


In [41]:
#-- Cluster 3
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park, Scarborough Town Centre, Wexford ...",2,Bakery,Gift Shop,Shipping Store,Wings Joint,Construction & Landscaping,Gym,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store


In [42]:
#-- Cluster 4
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,3,Korean Restaurant,Business Service,Park,Coffee Shop,Wings Joint,Discount Store,Gym,Grocery Store,Golf Course,Gift Shop
